In [42]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
# import seaborn as sns

from t import *


%matplotlib inline
sns.set_style("whitegrid")

#df= pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data.csv', low_memory=False)
df = pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data_new.csv', low_memory=False)

df1 = add_target_column(df)

# train_df , test_df = train_test_split(df1)

train_df = pd.read_csv('train_df_augmented.csv')
test_df = get_test_data(df1)

# train_df = perform_preprocessing(train_df)
test_df = perform_preprocessing(test_df)

/home/jupyter/r.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['FEAT1'] = pd.Series(perform_spell_correction_manual(df['FEAT1']), index=df.index)
/home/jupyter/r.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['FEAT1'] = pd.Series(perform_spell_correction_walmart(df['FEAT1']), index=df.index)
/home/jupyter/r.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [43]:
# train_df = data_augmentation_simple_walmart(train_df,n_per_original=2)

In [44]:
len(train_df), len(test_df)

(69937, 244187)

# TF calculations

In [45]:
X_train = train_df['FEAT1']
Y_train = train_df['TARGET']
X_test = test_df['FEAT1']

In [46]:
Y_train.value_counts().head()

AVOCADOES_AVOCADOES       1404
CORN_CORN                 1391
SNACKS_SNACKS             1263
APPLES_GALA               1211
CITRUS_ORANGES - OTHER    1163
Name: TARGET, dtype: int64

In [47]:
# AVOCADOES_AVOCADOES       1404
# CORN_CORN                 1391
# SNACKS_SNACKS             1263
# APPLES_GALA               1211
# CITRUS_ORANGES - OTHER    1163

In [48]:
pipe = pipeline_voting_hard(solver='lbfgs',penalty='l2', random_state=42 ,max_iter=100)

In [49]:
pipe

VotingClassifier(estimators=[('RF',
                              Pipeline(steps=[('vect',
                                               TfidfVectorizer(min_df=10)),
                                              ('clf',
                                               RandomForestClassifier(class_weight='balanced',
                                                                      n_estimators=220,
                                                                      random_state=42))])),
                             ('LR',
                              Pipeline(steps=[('vect',
                                               TfidfVectorizer(min_df=10)),
                                              ('clf',
                                               LogisticRegression(C=250,
                                                                  class_weight='balanced',
                                                                  random_state=42))]))])

In [50]:
%%time

import time
start = time.time()
pipe.fit(X_train, Y_train)
print(time.time()-start)

502.46101212501526
CPU times: user 9min 4s, sys: 6min 31s, total: 15min 36s
Wall time: 8min 22s


/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [51]:
Y_pred = pipe.predict(X_test)

In [52]:
pd.Series(Y_pred).value_counts().head()

CUT FRUIT PACKAGED_JAR FRUIT           16211
SNACKS_SNACKS                          14181
CUT FRUIT PROCESSED_VEGETABLE TRAYS    13815
APPLES_MISC                            11603
CITRUS_ORANGES - OTHER                  9630
dtype: int64

In [53]:
# CUT FRUIT PACKAGED_JAR FRUIT           16211
# SNACKS_SNACKS                          14181
# CUT FRUIT PROCESSED_VEGETABLE TRAYS    13815
# APPLES_MISC                            11603
# CITRUS_ORANGES - OTHER                  9630

In [54]:
test_df['Predicted_target'] = pd.Series(Y_pred, index= test_df.index)

/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [61]:
test_df.to_csv('test_df_with_model_predictions.csv', index=True)

In [63]:
test_df.to_csv('/data/DS_INTERN/data/RAW_DATA/test_df_with_model_predictions.csv', index=True)

In [13]:
# item1_desc_mapping(train_df, test_df)

In [64]:
%%time

major_df = fuzzy_tfidf_feat1(train_df, test_df, partial=False, ratio_type='sort')

CPU times: user 49min 26s, sys: 2min 11s, total: 51min 37s
Wall time: 51min 36s


In [65]:
# %%time

# FEAT1_TARGET_mapping_train = dict(zip(train_df.FEAT1, train_df.TARGET))

# ratio_threshold = 70

# import time 
# start = time.time()

# for row in range(len(major_df)):
#     data_row = major_df.iloc[row]
#     print(row)
#     if data_row['Ratio'] >=ratio_threshold:
# #         print(data_row['Ratio'])
#         test_df.iloc[row]['Predicted_target'] = FEAT1_TARGET_mapping_train[data_row['FEAT1_train']]
# #         print(time.time()-start)




In [66]:
test_df_dummy = test_df

In [67]:
final = test_df_dummy.reset_index(drop=True).join(major_df)

In [68]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [69]:
%%time
final['fuzzy_target'] = final.progress_apply(lambda row: FEAT1_TARGET_mapping_train[row['FEAT1_train']], axis=1)

A Jupyter Widget

CPU times: user 6.18 s, sys: 140 ms, total: 6.32 s
Wall time: 6.3 s


In [83]:
final['Final_Predicted_target'] = np.where((final['Ratio'] >= 50), final['fuzzy_target'], final['Predicted_target'])

In [84]:
def trust_factor(test_df):
    
    import Levenshtein as lev
    from fuzzywuzzy import fuzz
    
#    test_df['Predicted_target'] = pd.Series(Y_pred, index= test_df.index)
    test_df[['category','variety']] = test_df['Final_Predicted_target'].str.split('_', expand=True)
#    test_df.drop(['Predicted_target'], axis=  1, inplace = True)
    
    test_df['category'] = pd.Series(perform_spell_correction_manual(test_df['category']), index=test_df.index)
    test_df['variety'] = pd.Series(perform_spell_correction_manual(test_df['variety']), index=test_df.index)
    
    test_df['category'] = pd.Series(perform_spell_correction_walmart(test_df['category']), index=test_df.index)
    test_df['variety'] = pd.Series(perform_spell_correction_walmart(test_df['variety']), index=test_df.index)
    
    
    test_df['category'] = pd.Series(perform_lemmatization(test_df['category']), index=test_df.index)
    test_df['variety'] = pd.Series(perform_lemmatization(test_df['variety']), index=test_df.index)
    
    
    test_df['fuzzy_category'] = test_df.apply(lambda x: fuzz.partial_token_set_ratio(x['FEAT1'],x['category']), axis=1)
    test_df['fuzzy_variety'] = test_df.apply(lambda x: fuzz.partial_token_set_ratio(x['FEAT1'],x['variety']), axis=1)
    
    test_df['TF_cat'] = test_df.apply(lambda x: 1 if x['fuzzy_category']>=80 else 0, axis=1)
    test_df['TF_var'] = test_df.apply(lambda x: 1 if x['fuzzy_variety']>=80 else 0, axis=1)
    
    test_df['TF'] = np.where((test_df['TF_cat'] == 1) | (test_df['TF_var'] ==1),1, 0)
    
    return test_df

In [85]:
test_df_fuzzy = trust_factor(final)
test_df_fuzzy.describe()

,MDS_FAM_ID,UPC_NBR,DEPT_NBR,SALES_AMT_DOLLAR,NET_SHIP_COST_DOLLAR,SALES_QTY,CONSUMER_ITEM_NBR,REPL_GROUP_NBR,PRODUCT_NBR,Ratio,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,2.441870e+05,2.441870e+05,244187.000000,8.654800e+04,5.211600e+04,8.654800e+04,2.429830e+05,2.330900e+05,2.441340e+05,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000
mean,5.422204e+07,1.193594e+11,92.178691,1.467326e+05,2.199840e+05,6.414759e+04,9.569301e+06,1.031263e+08,9.562269e+06,59.887144,88.041435,90.088997,0.787990,0.814572,0.886280
std,1.766833e+07,1.866042e+11,9.161114,1.776146e+06,2.169458e+06,6.511948e+05,6.049217e+06,7.641933e+07,6.071109e+06,8.797121,23.798250,21.426107,0.408732,0.388645,0.317472
min,3.871000e+03,0.000000e+00,1.000000,-2.769620e+03,-2.889000e+04,-3.154176e+04,2.394000e+03,1.500000e+01,0.000000e+00,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.310606e+07,4.002031e+10,94.000000,6.867275e+00,1.029158e+03,3.000000e+00,8.197535e+06,1.547112e+07,8.080944e+06,54.000000,100.000000,100.000000,1.000000,1.000000,1.000000
50%,5.467994e+07,4.002102e+10,94.000000,9.378376e+02,6.548292e+03,3.523700e+02,9.505364e+06,1.336468e+08,9.548793e+06,59.000000,100.000000,100.000000,1.000000,1.000000,1.000000
75%,5.505387e+07,8.548690e+10,94.000000,1.285656e+04,3.781332e+04,6.486000e+03,1.065362e+07,1.835725e+08,1.065386e+07,64.000000,100.000000,100.000000,1.000000,1.000000,1.000000
max,2.558749e+08,8.002970e+12,95.000000,1.884090e+08,1.914145e+08,6.278640e+07,1.164568e+08,2.067939e+08,1.164568e+08,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000


In [76]:
test_df_fuzzy.to_csv('test_df_with_model_predictions_with_fuzzy.csv', index=True)

In [77]:
test_df_fuzzy.to_csv('/data/DS_INTERN/data/RAW_DATA/test_df_with_model_predictions_with_fuzzy.csv', index=True)

In [78]:
test_df_fuzzy.head()

,COUNTRY_CODE,SBU,SP_CLASS,MDS_FAM_ID,UPC_NBR,DEPT_NBR,DEPT_DESC,CATG_DESC,SUBCATG_DESC,FINELINE_DESC,...,Ratio,fuzzy_target,Final_Predicted_target,category,variety,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
0,US,FOOD,PRODUCE & FLOWERS,254412860,76101011282,94,PRODUCE,NUT MEAT,NUT WALNUT,WALNUT TRAY,...,63,NUT MEATS_SHELLED WALNUTS,NUT MEATS_SHELLED WALNUTS,NUT MEAT,SHELLED WALNUT,100,100,1,1,1
1,US,FOOD,PRODUCE & FLOWERS,162040632,80394430117,94,PRODUCE,MISC FLORAL,PR FLORAL,PR ORGANICOS,...,59,DRIED FRUIT_DRIED MANGOES,DRIED VEGGIES_DRIED SNAP PEAS,DRIED VEGGIE,DRIED SNAP PEA,100,100,1,1,1
2,US,FOOD,PRODUCE & FLOWERS,93643850,71631063457,94,PRODUCE,MISC FLORAL,HI AND MORE,PREMADE FRUIT BASKET,...,58,CUT FRUIT PROCESSED_SEASONAL BLEND,APPLES_MISC,APPLE,MISC,40,100,0,1,1
3,US,FOOD,PRODUCE & FLOWERS,93241707,71631063457,94,PRODUCE,MISC FLORAL,HI AND MORE,PREMADE FRUIT BASKET,...,58,CUT FRUIT PROCESSED_SEASONAL BLEND,APPLES_MISC,APPLE,MISC,40,100,0,1,1
4,US,FOOD,PRODUCE & FLOWERS,75494753,8185195601,94,PRODUCE,MISC FLORAL,HI AND MORE,PREMADE FRUIT BASKET,...,62,MIXED BOUQUETS_MIXED BOUQUETS,APPLES_MISC,APPLE,MISC,40,100,0,1,1


In [79]:
test_df_fuzzy[['Predicted_target','fuzzy_target']]

,Predicted_target,fuzzy_target
0,NUT MEATS_SHELLED WALNUTS,NUT MEATS_SHELLED WALNUTS
1,DRIED VEGGIES_DRIED SNAP PEAS,DRIED FRUIT_DRIED MANGOES
2,APPLES_MISC,CUT FRUIT PROCESSED_SEASONAL BLEND
3,APPLES_MISC,CUT FRUIT PROCESSED_SEASONAL BLEND
4,APPLES_MISC,MIXED BOUQUETS_MIXED BOUQUETS
5,APPLES_MISC,CUT FRUIT PROCESSED_SEASONAL BLEND
6,APPLES_MISC,MIXED BOUQUETS_MIXED BOUQUETS
7,APPLES_MISC,CUT FRUIT PROCESSED_FRUIT BOWL
8,APPLES_MISC,CUT FRUIT PROCESSED_SEASONAL BLEND
9,CONSUMER BUNCHES_CONSUMER BUNCHES,MIXED BOUQUETS_MIXED BOUQUETS


In [100]:
pipe['vect']

TypeError: 'VotingClassifier' object is not subscriptable

In [112]:
pipe.named_estimators_['LR']['vect'].get_feature_names()

['acc',
 'accent',
 'acorn',
 'add',
 'added',
 'additional',
 'adu',
 'agm',
 'agr',
 'air',
 'alcohol',
 'alcoholic',
 'alice',
 'almnd',
 'almond',
 'alms',
 'aloe',
 'also',
 'alstro',
 'alt',
 'ambrosia',
 'american',
 'amz',
 'anaheim',
 'angel',
 'animal',
 'anise',
 'anjou',
 'apbl',
 'api',
 'app',
 'appetiser',
 'appetizer',
 'apple',
 'apricot',
 'arg',
 'aril',
 'arizona',
 'arp',
 'arr',
 'arrangement',
 'art',
 'artichoke',
 'artisan',
 'arugula',
 'asb',
 'asian',
 'asp',
 'asparagus',
 'assorted',
 'asst',
 'asstd',
 'ataulfo',
 'athena',
 'aubergine',
 'autumn',
 'avm',
 'avocado',
 'avorh',
 'avrnh',
 'ayc',
 'ayco',
 'baby',
 'bac',
 'backpack',
 'backpacks',
 'backroom',
 'bacon',
 'bag',
 'bagel',
 'bagged',
 'bags',
 'baj',
 'bake',
 'baker',
 'baking',
 'balsamic',
 'ban',
 'banana',
 'banded',
 'barcode',
 'barley',
 'bart',
 'bartlett',
 'base',
 'based',
 'bash',
 'basil',
 'basket',
 'batata',
 'bauman',
 'bay',
 'bbq',
 'bbykle',
 'bbysp',
 'bch',
 'bean',
 

In [116]:
train_df[train_df['FEAT1']=='CUSTOMER ORDERED PROGRAM COP HYDRG BLUE CUSTOMER ORDER PROGRAM FLORAL CONSUMER BUNCH  SHOPPER BUNCH EVERYDAY CUSTOMER ORDER']

,ITEM1_DESC,FEAT1,TARGET
68510,CB COP HYDRG BLUE,CUSTOMER ORDERED PROGRAM COP HYDRG BLUE CUSTOM...,CONSUMER BUNCHES_CONSUMER BUNCHES
